In [2]:
%run ../processing/read_data.py
%run ../processing/functions.ipynb

import numpy as np
import pandas as pd
import os
from PIL import Image

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn import model_selection

In [3]:
from __future__ import absolute_import, division, print_function
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, ZeroPadding3D
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

1.13.1


In [4]:
path_to_read = '../../data/dl_marcelo_perc/'
files = []

for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.png' in file:
            files.append(os.path.join(r, file))
files.sort()
np_files = np.asarray(files)

In [5]:
conversion = {'IIL_' : 5, 'IIP_' : 7, '_II_' : 4, 'IIn_' : 6, '_Ia_' : 0, '_Ib_' : 1, 'Ibc_' : 2, '_Ic_' : 3}
conversion_bool = {'IIL_' : 0, 'IIP_' : 0, '_II_' : 0, 'IIn_' : 0, '_Ia_' : 1, '_Ib_' : 0, 'Ibc_' : 0, '_Ic_' : 0}


In [7]:
np_grouped = np_files.reshape(round(np_files.shape[0]/4), 4)
class_id = []
#CHANGE HERE THE INDEX EVERYTIME WE CHANGE THE NAME FROM THE PATH OF THE FOLDER
for g in np_grouped:
    class_id = class_id + [[g[0][-20:-12], g[0][-12:-8]]] 
types = []
types_bool = []
for ids in class_id:
    types = types + [conversion[ids[1]]]
    types_bool = types_bool + [conversion_bool[ids[1]]]
    
astr_objs = []
for i in range(len(np_grouped)):
    astr_objs = astr_objs + [class_id[i]]
    for file in np_grouped[i]:
        img = Image.open(file).convert('L')
        arr = np.array(img)
        astr_objs[-1] = astr_objs[-1] + [arr[5:35,10:56]] 
        #CHANGE HERE IF WE EVENTUALLY CHANGE THE SIZE OF THE IMAGE, OR WANT TO CUT OTHER FRAMES

print( "The quantity of objects is: ", len(astr_objs))

ValueError: cannot reshape array of size 84954 into shape (21238,4)

# K-FOLD

In [ ]:
labels = []
images = []
ids = []
for o in astr_objs:
    labels.append((conversion_bool[o[1]])) #converting here to numbers to assure label in training
    images.append(o[2:])
    ids.append(o[0])
labels = np.asarray(labels)
images = np.asarray(images)
ids = np.asarray(ids)

Xb = images
yb = labels
kfb = KFold(n_splits=5)
Xb_train = []
Xb_test = []
yb_train = []
yb_test = []
id_train = []
id_test = []
for train_index, test_index in kfb.split(Xb):
    print("TRAIN:", train_index, "TEST:", test_index)
    Xb_train.append(Xb[train_index])
    Xb_test.append(Xb[test_index])
    yb_train.append(yb[train_index])
    yb_test.append(yb[test_index])
    id_train.append(ids[train_index])
    id_test.append(ids[test_index])

In [ ]:
print('The shapes of the objects are:')
print('ids: ', ids.shape)
print('labels: ', labels.shape)
print('images: ', images.shape)
print('train_images: ', Xb_train[0].shape)
print('Other useful object: "test_images"')

In [ ]:
#This cell bugs the pc a little
#Normalization and separating the folds to the models
train_images = np.around(Xb_train[0]/255, decimals = 2)
test_images = np.around(Xb_test[0]/255, decimals = 2)

train_swaped = np.swapaxes(train_images,1,3)
test_swaped = np.swapaxes(test_images,1,3)



train_images4 = np.around(Xb_train[4]/255, decimals = 2)
test_images4 = np.around(Xb_test[4]/255, decimals = 2)

train_swaped4 = np.swapaxes(train_images4,1,3)
test_swaped4 = np.swapaxes(test_images4,1,3)

# Model Convolution NN 2D

In [ ]:
np.random.seed(9)
img_rows, img_cols = 30, 46
input_shape = (img_cols, img_rows, 4)

model_cv = Sequential()

model_cv.add(Conv2D(64, kernel_size=(4,4), padding='same', activation='relu', input_shape=input_shape))
model_cv.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=input_shape))
model_cv.add(MaxPooling2D(pool_size=(2,2)))
model_cv.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=input_shape))
model_cv.add(MaxPooling2D(pool_size=(2,2)))


model_cv.add(Flatten())
model_cv.add(Dense(8, activation=tf.nn.softmax))

model_cv.summary()

In [ ]:
np.random.seed(9)
model_cv.compile(optimizer='adam', #optimizer = 'adadelta'
              loss='sparse_categorical_crossentropy', #loss = "binary_crossentropy"
              metrics=['accuracy'])

In [ ]:
np.random.seed(9)
model_cv.fit(train_swaped, yb_train[0], #validation_split = 0.1
             epochs=10) 

### With Kfold 4

In [ ]:
np.random.seed(9)
img_rows, img_cols = 30, 46
input_shape = (img_cols, img_rows, 4)

model_cv4 = Sequential()

model_cv4.add(Conv2D(64, kernel_size=(4,4), padding='same', activation='relu', input_shape=input_shape))
model_cv4.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=input_shape))
model_cv4.add(MaxPooling2D(pool_size=(2,2)))
model_cv4.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=input_shape))
model_cv4.add(MaxPooling2D(pool_size=(2,2)))


model_cv4.add(Flatten())
model_cv4.add(Dense(8, activation=tf.nn.softmax))

model_cv4.summary()

In [ ]:
np.random.seed(9)
model_cv4.compile(optimizer='adam', #optimizer = 'adadelta'
              loss='sparse_categorical_crossentropy', #loss = "binary_crossentropy"
              metrics=['accuracy'])

In [ ]:
np.random.seed(9)
model_cv4.fit(train_swaped4, yb_train[4], #validation_split = 0.1
             epochs=10) 

# Evaluation

In [ ]:
test_loss_cv, test_acc_cv = model_cv.evaluate(test_swaped, yb_test[0])

print('Test accuracy model_cv:', test_acc_cv)
print('Test loss model_cv:', test_loss_cv)

In [ ]:
test_loss_cv4, test_acc_cv4 = model_cv4.evaluate(test_swaped4, yb_test[4])

print('Test accuracy model_cv:', test_acc_cv4)
print('Test loss model_cv:', test_loss_cv4)